In [ ]:
import cv2 as cv
import numpy as np
import math
import sys
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/ForAllOneK/Original'
data = pd.read_csv(os.path.join(path, 'cost.csv'))

In [ ]:
data.head()

,file_name,cost
0,00000002.jpg,138.969146
1,00000003.jpg,26.790113
2,00000005.jpg,39.446019
3,00000004.jpg,78.709447
4,00000006.jpg,34.586774


In [ ]:
thresh_vals = data.describe()['cost']
thresh_vals = thresh_vals.to_dict()
thresh_vals['median'] = data.median()[0]
thresh_vals['mode']= 3*data.median()[0] - 2*data.mean()[0]
thresh_vals.pop('count')

1000.0

In [ ]:
"""file_name = []
for f in tqdm(os.listdir(path)):
    if not f.endswith('.jpg'):
        continue
    if os.path.isdir(os.path.join(path, f)):
        continue
    file_name.append(f)
df = pd.DataFrame()
df['file_name'] = file_name
df['k_val'] = -1
df['cost_val'] = -1
#df['thresh_val_with_given_k'] = -1
df.to_csv(os.path.join(path,'MeanAsThresh.csv'), index = False)"""

100%|██████████| 1001/1001 [00:00<00:00, 6196.86it/s]


In [ ]:
def KMEANS(image, n_clusters):
    X = image.reshape(-1, 3)
    X = X/255.0
    km = KMeans(n_clusters=n_clusters)
    km.fit(X)
    return km.inertia_

In [ ]:
MEAN = thresh_vals['mean']
MEAN

67.31697583691374

In [ ]:
#df = pd.read_csv(os.path.join(path, 'data.csv'))
for f in tqdm(df['file_name']):
    #df = pd.read_csv(os.path.join(path, 'data.csv'))
    if not f.endswith('.jpg'):
        continue
    if os.path.isdir(os.path.join(path, f)):
        continue
    if int(df[df['file_name']==f]['k_val']) == -1:
        img = cv.imread(os.path.join(path,f))
        image_g = cv.GaussianBlur(img,(5,5),0)
        image_g = cv.GaussianBlur(image_g, (5,5), 0)
        image_g = cv.GaussianBlur(image_g, (5,5), 0)
        for k in range(10,300,5):
            cost_val = KMEANS(image_g, k)
            if cost_val<=MEAN:
                break
        df.loc[df.file_name == f, 'k_val'] = k
        df.loc[df.file_name == f, 'cost_val'] = cost_val
        #print(f,k,cost_val)
        #df.loc[df.file_name == f, 'thresh_val_with_given_k'] = cost_val
        #df.to_csv(os.path.join(path,'data.csv'), index = False)
    else:
        continue


100%|██████████| 225/225 [1:47:40<00:00, 28.71s/it]
